<a href="https://colab.research.google.com/github/EygonZhang/FX-BEMS/blob/main/FX_BEMS%C2%A9_(Education_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Design the geometry input

In [1]:
import numpy as np
import math

# Set the range and distribution of r
num_elements = 10  # Number of blade elements
R = 8.5
r_start = 2.473  # Starting radius (m)
r_end = 8.5    # Ending radius (m)
c_start = 1.545 # Starting chord length (m)
c_end = 0.927 # Ending chord length (m)
# Generate equally spaced r values using linspace, creating num_elements + 1 points divided into num_elements segments
delta_r = (r_end - r_start) / num_elements  # Width of each segment δr
radii = np.linspace(r_start, r_end, num_elements + 1)
chord_lengths = np.linspace(c_start, c_end, num_elements + 1)

theta_root = math.radians(10)
theta_tip = math.radians(0)

def get_twist_angle(r, r_start, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * ((r - r_start) / (R - r_start))

print("r (m)    twist (deg)")
for r in radii:
    twist = get_twist_angle(r, r_start, R, theta_root, theta_tip)
    print(f"{r:6.3f}   {math.degrees(twist):6.2f}")
print("Segment width δr:", delta_r)
print("Radius values radii:", radii)
print("Chord values chord:", chord_lengths)

r (m)    twist (deg)
 2.473    10.00
 3.076     9.00
 3.678     8.00
 4.281     7.00
 4.884     6.00
 5.486     5.00
 6.089     4.00
 6.692     3.00
 7.295     2.00
 7.897     1.00
 8.500     0.00
Segment width δr: 0.6027
Radius values radii: [2.473  3.0757 3.6784 4.2811 4.8838 5.4865 6.0892 6.6919 7.2946 7.8973
 8.5   ]
Chord values chord: [1.545  1.4832 1.4214 1.3596 1.2978 1.236  1.1742 1.1124 1.0506 0.9888
 0.927 ]


1. Calculate a and a' via BEM **iteration** with XFoil input

In [2]:
import numpy as np

# Parameter settings
U_infinity = 2  # Freestream velocity (m/s)
omega = 1   # Rotation speed (rad/s)
R = 8.5  # Fixed total blade radius (m)
B = 3  # Number of blades
tolerance = 1e-6  # Iteration convergence tolerance
small_value = 1e-6  # Small value to prevent division by zero errors

# Initial CL, CD
#================== The CL and CD are prompted from XFoil 2D results without considering the 3D rotation correction==================#
CL_initial = np.array([0.00000, 0.11060, 0.22060, 0.32930, 0.43620, 0.54070, 0.64220, 0.75230, 0.87660, 0.99990, 1.07670])
CD_initial = np.array([0.00096, 0.00097, 0.00100, 0.00106, 0.00115, 0.00127, 0.00141, 0.00163, 0.00192, 0.00233, 0.00300])

# Twist angle parameters
theta_root = math.radians(10)  # Twist angle at blade root (10°)
theta_tip = math.radians(0)   # Twist angle at blade tip (0°)

# Calculate twist angle at given radius position

def get_twist_angle(r, r_start, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * ((r - r_start) / (R - r_start))

# Prandtl tip loss factor calculation function
def prandtl_tip_loss(B, r, R, phi):
    if abs(r - R) < small_value:
        return 0.35  # When r = R, F = 0.25
    f = (B / 2) * (R - r) / (r * math.sin(phi))
    F = (2 / math.pi) * math.acos(math.exp(-f))
    return F

# Function to create T transpose matrix
def calculate_forces_projection(CL, CD, phi):
    # Calculate T matrix and apply to [CL, CD]
    T = np.array([[math.cos(phi), math.sin(phi)],
                  [math.sin(phi), -math.cos(phi)]])
    forces = np.array([CL, CD])
    projected_forces = T @ forces  # Matrix multiplication
    Cn, Ct = projected_forces
    return Cn, Ct

# Function to calculate induction factors, considering twist angle and new formulas
def calculate_induction_with_twist(a_init, a_prime_init, r, chord, CL, CD, consider_tip_loss):
    a = a_init
    a_prime = a_prime_init

    for _ in range(1000):
        # Calculate twist angle and correct relative flow angle
        twist_angle = get_twist_angle(r, r_start, R, theta_root, theta_tip)
        phi = math.atan((1 - a) * U_infinity / ((1 + a_prime) * omega * r + small_value)) - twist_angle

        # Calculate Cn and Ct based on corrected phi value
        Cn, Ct = calculate_forces_projection(CL, CD, phi)

        # Calculate Prandtl tip loss factor
        if consider_tip_loss:
            F = prandtl_tip_loss(B, r, R, phi)
        else:
            F = 1  # Without considering tip loss

        # Calculate Solidity (blade loading coefficient) sigma
        sigma = chord / (2 * math.pi * r)

        # Update a and a_prime
        a_new = 1 / (1 + F * (4 * math.sin(phi)**2) / (sigma * Cn + small_value))
        a_prime_new = 1 / (F * (4 * math.sin(phi) * math.cos(phi) / (sigma * Ct + small_value)) - 1)

        # Check convergence condition
        if abs(a_new - a) < tolerance and abs(a_prime_new - a_prime) < tolerance:
            a, a_prime = a_new, a_prime_new
            break
        # Update induction factors
        a, a_prime = a_new, a_prime_new

    return a, a_prime, F, Cn, Ct

# Initialize result arrays
a_no_tip = []
a_prime_no_tip = []
a_with_tip = []
a_prime_with_tip = []
F_values = []
Cn_initial = []
Ct_initial = []

# Calculate induction factors and tip loss factors for each radius value
for i, r in enumerate(radii):
    CL = CL_initial[i]
    CD = CD_initial[i]
    chord = chord_lengths[i]

    # Calculate induction factors without tip loss
    a_nt, a_prime_nt, _, Cn, Ct = calculate_induction_with_twist(0.3, 0.01, r, chord, CL, CD, consider_tip_loss=False)
    a_no_tip.append(a_nt)
    a_prime_no_tip.append(a_prime_nt)

    # Calculate induction factors with tip loss
    a_wt, a_prime_wt, F, Cn, Ct = calculate_induction_with_twist(0.3, 0.01, r, chord, CL, CD, consider_tip_loss=True)
    a_with_tip.append(a_wt)
    a_prime_with_tip.append(a_prime_wt)
    F_values.append(F)
    Cn_initial.append(Cn)
    Ct_initial.append(Ct)

# Output results
print("Radius (r)  | a (no tip loss) | a' (no tip loss) | a (with tip loss) | a'(with tip loss) | Tip loss F |    Cn    |    Ct    |    CL   |   CD  ")
for i, r in enumerate(radii):
    print(f"{r:.3f}     | {a_no_tip[i]:.6f}      | {a_prime_no_tip[i]:.6f}       | "
          f"{a_with_tip[i]:.6f}       | {a_prime_with_tip[i]:.6f}      | {F_values[i]:.4f}    | "
          f"{Cn_initial[i]:.6f} | {Ct_initial[i]:.6f} | {CL_initial[i]:.4f}  | {CD_initial[i]:.4f}")

Radius (r)  | a (no tip loss) | a' (no tip loss) | a (with tip loss) | a'(with tip loss) | Tip loss F |    Cn    |    Ct    |    CL   |   CD  
2.473     | 0.000050      | -0.000049       | 0.000050       | -0.000049      | 0.9997    | 0.000465 | -0.000840 | 0.0000  | 0.0010
3.076     | 0.011972      | 0.002276       | 0.011982       | 0.002278      | 0.9991    | 0.101692 | 0.043496 | 0.1106  | 0.0010
3.678     | 0.027123      | 0.003573       | 0.027178       | 0.003580      | 0.9981    | 0.207908 | 0.073754 | 0.2206  | 0.0010
4.281     | 0.044740      | 0.004324       | 0.044936       | 0.004341      | 0.9960    | 0.315299 | 0.095006 | 0.3293  | 0.0011
4.884     | 0.063217      | 0.004749       | 0.063828       | 0.004789      | 0.9916    | 0.421859 | 0.110935 | 0.4362  | 0.0011
5.486     | 0.080176      | 0.004960       | 0.081936       | 0.005052      | 0.9817    | 0.526238 | 0.124225 | 0.5407  | 0.0013
6.089     | 0.093034      | 0.005024       | 0.097767       | 0.005234      | 0.

2. Tip Loss Factor F check

In [4]:
import math
import numpy as np

theta_root = math.radians(10)  # Twist angle at blade root (10°)
theta_tip = math.radians(0)    # Twist angle at blade tip (0°)

# Data for r = R position
r = R
chord = 0.927
CL = 1.07670
CD = 0.00300

def get_twist_angle(r, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * (r / R)

def prandtl_tip_loss(B, r, R, phi):
    # When r approaches R, directly return assumed F=0.5
    if abs(r - R) < small_value:
        return 0.25
    f = (B / 2) * (R - r) / (r * math.sin(phi))
    F = (2 / math.pi) * math.acos(math.exp(-f))
    return F


def calculate_induction_with_twist_debug(a_init, a_prime_init, r, chord, CL, CD, consider_tip_loss):
    a = a_init
    a_prime = a_prime_init
    print(f"【Debug info】 r = {r:.3f} m, chord = {chord:.3f} m, CL = {CL}, CD = {CD}")
    for it in range(1000):
        twist_angle = get_twist_angle(r, R, theta_root, theta_tip)
        # Calculate relative flow angle phi (subtracting twist angle)
        phi = math.atan((1 - a) * U_infinity / ((1 + a_prime) * omega * r + small_value)) - twist_angle

        # When considering tip loss, r==R directly returns F=0.5
        if consider_tip_loss:
            F = prandtl_tip_loss(B, r, R, phi)
        else:
            F = 1

        Cn, Ct = calculate_forces_projection(CL, CD, phi)
        sigma = chord / (2 * math.pi * r)

        # Update a and a′ formulas
        a_new = 1 / (1 + F * (4 * math.sin(phi)**2) / (sigma * Cn + small_value))
        a_prime_new = 1 / (F * (4 * math.sin(phi) * math.cos(phi) / (sigma * Ct + small_value)) - 1)

        print(f"Iteration {it:2d}: φ = {phi:.6f} rad, a = {a_new:.6f}, a' = {a_prime_new:.6f}, F = {F:.4f}")

        # Convergence check
        if abs(a_new - a) < tolerance and abs(a_prime_new - a_prime) < tolerance:
            a, a_prime = a_new, a_prime_new
            break
        a, a_prime = a_new, a_prime_new

    return a, a_prime, F, Cn, Ct

print("----- Debug process with tip loss (F=0.5 when r=R) -----")
a_with_tip, a_prime_with_tip, F_with_tip, Cn_with_tip, Ct_with_tip = calculate_induction_with_twist_debug(
    0.3, 0.01, r, chord, CL, CD, consider_tip_loss=True
)
print("\nConvergence result (with tip loss): a = {:.6f}, a' = {:.6f}, F = {:.4f}".format(a_with_tip, a_prime_with_tip, F_with_tip))

----- Debug process with tip loss (F=0.5 when r=R) -----
【Debug info】 r = 8.500 m, chord = 0.927 m, CL = 1.0767, CD = 0.003
Iteration  0: φ = 0.161652 rad, a = 0.416022, a' = 0.018971, F = 0.2500
Iteration  1: φ = 0.134040 rad, a = 0.509201, a' = 0.018823, F = 0.2500
Iteration  2: φ = 0.112867 rad, a = 0.594231, a' = 0.018698, F = 0.2500
Iteration  3: φ = 0.093450 rad, a = 0.681283, a' = 0.018561, F = 0.2500
Iteration  4: φ = 0.073493 rad, a = 0.775676, a' = 0.018375, F = 0.2500
Iteration  5: φ = 0.051783 rad, a = 0.874492, a' = 0.018047, F = 0.2500
Iteration  6: φ = 0.029000 rad, a = 0.956938, a' = 0.017227, F = 0.2500
Iteration  7: φ = 0.009960 rad, a = 0.994720, a' = 0.013748, F = 0.2500
Iteration  8: φ = 0.001226 rad, a = 0.999920, a' = -0.022468, F = 0.2500
Iteration  9: φ = 0.000019 rad, a = 1.000000, a' = -0.723892, F = 0.2500
Iteration 10: φ = 0.000000 rad, a = 1.000000, a' = -0.999666, F = 0.2500
Iteration 11: φ = 0.000000 rad, a = 1.000000, a' = -1.000000, F = 0.2500
Iteratio

3. Calculate the Relative velocity **Urel** and Flow angle **phi** (Private)

3. Output the Lift & Drag and Normal & Tangential Force (Private)

4. CSV output (Private)
